In [64]:
from selenium import webdriver         
from selenium.webdriver.common.by import By    
import time                                   
from bs4 import BeautifulSoup                 
import pandas as pd                             
import re                                       
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np

In [19]:
# Start a new Chrome web driver
driver = webdriver.Chrome(ChromeDriverManager().install())
# Go to website
driver.get("https://www.aclu.org/legislative-attacks-on-lgbtq-rights?impact=&state=")
# Locate scrollable element containing table
element = driver.find_element(By.XPATH, "/html/body/div/div[4]/div/section[1]/div/div/div[2]/div[4]/div")
table_html = element.get_attribute("innerHTML")

In [27]:
soup = BeautifulSoup(table_html, "html.parser")
rows = soup.find_all("tr")
rows.pop(0)

<tr class="data-table__header" data-v-7fd76bba=""><th data-v-7fd76bba="">
            state
            <div class="info-tooltip" data-v-507c9bbc="" data-v-7fd76bba=""><span class="info-tooltip__trigger icon info" data-v-507c9bbc="">i</span> <div class="info-tooltip__tip is-size-7 p-sm opens-right" data-v-507c9bbc=""><p data-v-507c9bbc="" data-v-7fd76bba="">
                Just like Congress at the federal level, every state has a legislative body responsible for passing
                laws that impact their state residents. While some states have been at the forefront of advancing
                LGBTQ equality, state-level attacks on LGBTQ people, particularly transgender people, have escalated
                dramatically since 2015.
              </p> <p data-v-507c9bbc="" data-v-7fd76bba=""><span class="highlight" data-v-507c9bbc="" data-v-7fd76bba="">
                  Click on a state in the map or table to show all the bills in that state.
                </span></p></div></d

In [42]:
rows

[<tr data-v-7fd76bba=""><td class="bill-state p-none" data-v-7fd76bba=""><a class="filter-hover columns is-mobile no-underline is-align-items-center" data-v-7fd76bba=""><div class="has-state-icon column mt-xs ml-xs mr-sm p-none has-state-icon__AK" data-v-7fd76bba=""></div> <div class="pl-none" data-v-7fd76bba="">Alaska</div></a></td> <td class="p-none" data-v-7fd76bba=""><a class="is-block-mobile mb-sm-mobile" data-v-7fd76bba="" href="https://www.akleg.gov/basis/Bill/Detail/33?Root=HB27#tab1_4" rel="noopener noreferrer" target="_blank">
               HB 27
             </a> <!-- --></td> <td class="bill-impact pb-xs" data-v-7fd76bba=""><a class="filter-hover is-block mb-xxs" data-v-7fd76bba=""><div class="mr-xxs slt-pill slt-pill__school" data-v-7fd76bba="">
                 Schools &amp; Education
               </div></a></td> <td class="p-none" data-v-7fd76bba=""><div class="bill-status is-size-6" data-v-7fd76bba="">Advancing</div> <div class="bill-status-description is-size-7" dat

In [28]:
row_soup = BeautifulSoup(str(rows), "html.parser")

In [43]:
rows[0]

<tr data-v-7fd76bba=""><td class="bill-state p-none" data-v-7fd76bba=""><a class="filter-hover columns is-mobile no-underline is-align-items-center" data-v-7fd76bba=""><div class="has-state-icon column mt-xs ml-xs mr-sm p-none has-state-icon__AK" data-v-7fd76bba=""></div> <div class="pl-none" data-v-7fd76bba="">Alaska</div></a></td> <td class="p-none" data-v-7fd76bba=""><a class="is-block-mobile mb-sm-mobile" data-v-7fd76bba="" href="https://www.akleg.gov/basis/Bill/Detail/33?Root=HB27#tab1_4" rel="noopener noreferrer" target="_blank">
              HB 27
            </a> <!-- --></td> <td class="bill-impact pb-xs" data-v-7fd76bba=""><a class="filter-hover is-block mb-xxs" data-v-7fd76bba=""><div class="mr-xxs slt-pill slt-pill__school" data-v-7fd76bba="">
                Schools &amp; Education
              </div></a></td> <td class="p-none" data-v-7fd76bba=""><div class="bill-status is-size-6" data-v-7fd76bba="">Advancing</div> <div class="bill-status-description is-size-7" data-v-7

In [63]:
rows[19]

<tr data-v-7fd76bba=""><td class="bill-state p-none" data-v-7fd76bba=""><a class="filter-hover columns is-mobile no-underline is-align-items-center" data-v-7fd76bba=""><div class="has-state-icon column mt-xs ml-xs mr-sm p-none has-state-icon__AR" data-v-7fd76bba=""></div> <div class="pl-none" data-v-7fd76bba="">Arkansas</div></a></td> <td class="p-none" data-v-7fd76bba=""><span class="is-block-mobile" data-v-7fd76bba="">HB 1468</span> <!-- --></td> <td class="bill-impact pb-xs" data-v-7fd76bba=""><a class="filter-hover is-block mb-xxs" data-v-7fd76bba=""><div class="mr-xxs slt-pill slt-pill__school" data-v-7fd76bba="">
                Schools &amp; Education
              </div></a></td> <td class="p-none" data-v-7fd76bba=""><div class="bill-status is-size-6" data-v-7fd76bba="">Advancing</div> <div class="bill-status-description is-size-7" data-v-7fd76bba="">
              Passed House; Senate second read and referred to committee, 03/27/2023
            </div></td></tr>

In [109]:
states, links, names, categories, statuses = [], [], [], [], []
cat_class_names = ["mr-xxs slt-pill slt-pill__school", "mr-xxs slt-pill slt-pill__speech", "mr-xxs slt-pill slt-pill__health",
    "mr-xxs slt-pill slt-pill__other", "mr-xxs slt-pill slt-pill__public", "mr-xxs slt-pill slt-pill__rights", "mr-xxs slt-pill slt-pill__ids"]
for i in range(len(rows)):
    specific_bill_data = rows[i]
    states.append(specific_bill_data.find("div", class_ = "pl-none").text)
    url_tag = specific_bill_data.find("a", class_ = "is-block-mobile mb-sm-mobile")
    if url_tag == None: 
        links.append(np.NAN)
        finding_name = specific_bill_data.find("span", class_ = "is-block-mobile")
        names.append(finding_name.text.strip())
    else: 
        links.append(url_tag["href"])
        names.append(url_tag.text.strip())
    for c_name in cat_class_names:
        cat_element = specific_bill_data.find("div", class_ = c_name)
        if cat_element == None: continue
        else: 
            categories.append(cat_element.text.strip())
            break
    status_tag = specific_bill_data.find("div", class_ = "bill-status-description is-size-7")
    statuses.append(status_tag.text.strip())
    

In [110]:
data = {"state": states, "link": links, "bill_name": names, "category": categories, "status": statuses}
df = pd.DataFrame(data)

In [112]:
df

,state,link,bill_name,category,status
0,Alaska,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB 27,Schools & Education,"Referred to committee, 01/19/2023"
1,Alaska,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB 105,Schools & Education,"First read and referred to committee, 03/08/2023"
2,Alaska,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,SB 96,Schools & Education,"First read and referred to committee, 03/08/2023"
3,Arizona,https://apps.azleg.gov/BillStatus/BillOverview...,SB 1028,Free Speech & Expression,"Passed Senate; House second read, 03/22/2023"
4,Arizona,https://apps.azleg.gov/BillStatus/BillOverview...,SB 1026,Free Speech & Expression,"Passed Senate; House second read, 03/09/2023"
...,...,...,...,...,...
429,Wyoming,https://wyoleg.gov/Legislation/2023/SF0111,SF 111,Healthcare,Passed Senate; House did not consider for intr...
430,Wyoming,https://wyoleg.gov/Legislation/2023/SF0117,SF 117,Schools & Education,Passed Senate; House did not consider for intr...
431,Wyoming,https://wyoleg.gov/Legislation/2023/HB0262,HB 262,Civil Rights,"Died in committee, 02/07/2023"
432,Wyoming,https://wyoleg.gov/Legislation/2023/SF0133,SF 133,Schools & Education,"Became act without Governor’s signature, 03/17..."


In [ ]:
df.to_csv('aclu_bill_data.csv', index = False)